# Blockchain Coins

### Building a blockchain coin form scratch in Python

It is recommented that the reader has read Blockchain fundamentals, as the bisac structure is similar just with an appopriat vocabulary. Furthermore, there are a few additions worth explaining.

### Proof of work/mining:

### Transactions (peer to peer):

### Transactions (peer to peer):

In [1]:
import time

In [ ]:
class Event:  # The Block
    def __init__(self, transaction, prev_hash=None):
        self.transaction = transaction
        self.timestamp = time.time()
        self.prev_hash = prev_hash
        self.hash = self.hashing()
        
    def hashing(self):
        return hash((self.transaction, self.timestamp, self.prev_hash))
    

class Transaction:
    def __init__(self, amount, to_id, from_id):
        self.to_id = to_id
        self.from_id = from_id
        self.amount = amount
    
    
class Coin:  # The BlockChain
    def __init__(self):
        self.history = [self.create_gen_block()]
        
    def create_gen_block(self):
        return Event('Genesis Event')
    
    def add_block(self, data):
        self.chain.append(Event(data, self.chain[-1].hash))
        
    def is_valid(self):
        for i in range(1, len(self.chain)):
            prev = self.chain[i-1]
            current = self.chain[i]
            
            if current.hash != current.hashing():
                return False
            
            if current.prev_hash != prev.hashing():
                return False
        
        return True
    
    def get_balance(id_address):
        balance = 0
        
        for 
        
    
    def view_chain(self):
        for block in self.chain:
            print(vars(block))